## Finding the Pacers latest game

In [1]:
# Get the Pacers team_id
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# Select the dictionary for the Pacers which contains their team ID
pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

pacers_id: 1610612754


In [2]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id, season_nullable=Season.default, season_type_nullable=SeasonType.default)

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

Searching through 82 game(s) for the game_id of 0022301188 where IND vs. ATL


## Retrieving the play by play data

In [3]:
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022301188,2,12,0,1,1:12 PM,12:00,None,Start of 1st Period (1:12 PM EST),None,None,None
1,0022301188,4,10,0,1,1:12 PM,12:00,Jump Ball Turner vs. Fernando: Tip to Hunter,None,None,None,None
2,0022301188,8,2,5,1,1:12 PM,11:38,Turner BLOCK (1 BLK),None,MISS Fernando 3' Layup,None,None
3,0022301188,10,4,0,1,1:12 PM,11:35,Turner REBOUND (Off:0 Def:1),None,None,None,None
4,0022301188,11,2,1,1,1:12 PM,11:28,MISS Haliburton 24' 3PT Jump Shot,None,None,None,None


In [4]:
print(f'EVENTMSGTYPE: {sorted(df.EVENTMSGTYPE.unique())}')

EVENTMSGTYPE: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]


In [5]:
from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [6]:
df.loc[df['EVENTMSGTYPE'] == 1].head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
9,0022301188,17,1,98,1,1:13 PM,11:02,Turner 4' Cutting Layup Shot (2 PTS) (Haliburt...,None,None,0 - 2,2
14,0022301188,25,1,76,1,1:14 PM,10:44,Siakam 3' Running Finger Roll Layup (2 PTS) (H...,None,None,0 - 5,5
17,0022301188,30,1,1,1,1:15 PM,10:12,Turner 28' 3PT Jump Shot (6 PTS) (Haliburton 3...,None,None,0 - 8,8
23,0022301188,39,1,41,1,1:19 PM,9:45,None,None,Murray 3' Running Layup (2 PTS),2 - 8,6
24,0022301188,40,1,1,1,1:19 PM,9:34,Turner 27' 3PT Jump Shot (9 PTS) (Haliburton 4...,None,None,2 - 11,9


In [7]:
emt_df = df.loc[df['EVENTMSGTYPE'] == 1]
print(f'EVENTMSGACTIONTYPE: {sorted(emt_df.EVENTMSGACTIONTYPE.unique())}')

EVENTMSGACTIONTYPE: [1, 5, 6, 7, 9, 41, 44, 47, 50, 52, 63, 71, 72, 74, 75, 76, 78, 79, 80, 86, 97, 98, 99, 100, 101, 103, 106, 107, 108]


## EVENTMSGTYPE == 1

In [14]:
import re
import operator

# The following expresion is specific to EventMsgType 1
p = re.compile('(\s{2}|\' )([\w+ ]*)')

# Get the PlayByPlay data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

# Declare a few variables
description = ''
event_msg_action_types = {}

# Loop over the play by play data
for play in plays:
  if play['EVENTMSGTYPE'] == 1:
    description = play['HOMEDESCRIPTION'] if play['HOMEDESCRIPTION'] is not None else play['VISITORDESCRIPTION']
    if description is not None:
      found = p.search(description)
      if found is not None:
        # Do a bit of searching and a little character magic: underscores and uppercase
        event_msg_action = re.sub(' ', '_', found.groups()[1].rstrip()).upper()
        # Add it to our dictionary
        event_msg_action_types[event_msg_action] = play['EVENTMSGACTIONTYPE']

# Sort it all
event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

for action in event_msg_action_types:
  print(f'\t{action[0]} = {action[1]}')

	3PT_JUMP_SHOT = 1
	3PT_PULLUP_JUMP_SHOT = 79
	3PT_STEP_BACK_JUMP_SHOT = 80
	ALLEY_OOP_DUNK = 52
	CUTTING_DUNK_SHOT = 108
	CUTTING_FINGER_ROLL_LAYUP_SHOT = 99
	CUTTING_LAYUP_SHOT = 98
	DRIVING_DUNK = 9
	DRIVING_FINGER_ROLL_LAYUP = 75
	DRIVING_FLOATING_JUMP_SHOT = 101
	DRIVING_LAYUP = 6
	DUNK = 7
	FADEAWAY_JUMPER = 63
	FINGER_ROLL_LAYUP = 71
	FLOATING_JUMP_SHOT = 78
	JUMP_SHOT = 1
	LAYUP = 5
	PULLUP_JUMP_SHOT = 79
	PUTBACK_LAYUP = 72
	REVERSE_LAYUP = 44
	RUNNING_ALLEY_OOP_DUNK_SHOT = 106
	RUNNING_ALLEY_OOP_LAYUP_SHOT = 100
	RUNNING_DUNK = 50
	RUNNING_FINGER_ROLL_LAYUP = 76
	RUNNING_LAYUP = 41
	RUNNING_PULL = 103
	RUNNING_REVERSE_LAYUP = 74
	STEP_BACK_JUMP_SHOT = 80
	TURNAROUND_FADEAWAY = 86
	TURNAROUND_JUMP_SHOT = 47


## EVENTMSGTYPE == 2

In [18]:
import re
import operator

# The following expression is specific to EventMsgType 1
p = re.compile('(\s{2}|\' )([\w+ ]*)')

# Get the Play By Play data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

# Declare a few variables 
description = ''
event_msg_action_types = {}

# Loop over the play by play data
# Do a bit of findall(regex) and a little character magic: underscores and uppercase
# We're using findall here as we have to deal with the extra word MISS at the beginning of the text.
# That extra text means we'll have multiple matches for our regex.
for play in plays:
  if play['EVENTMSGTYPE'] == 2:
    match = list()
    if play['HOMEDESCRIPTION'] is not None:
      match = p.findall(play['HOMEDESCRIPTION'])
    
    if not match:
      visitor_play = play['VISITORDESCRIPTION']
      if visitor_play is not None:
        match = p.findall(play['VISITORDESCRIPTION'])
    
    if match:
      event_msg_action = re.sub(' ', '_', match[0][1]).upper()
      event_msg_action_types[event_msg_action] = play['EVENTMSGACTIONTYPE']

event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

for action in event_msg_action_types:
  print(f'\t{action[0]} = {action[1]}')

	3PT_JUMP_SHOT = 1
	3PT_PULLUP_JUMP_SHOT = 79
	3PT_RUNNING_JUMP_SHOT = 2
	3PT_RUNNING_PULL = 103
	3PT_STEP_BACK_JUMP_SHOT = 80
	CUTTING_LAYUP_SHOT = 98
	DRIVING_FINGER_ROLL_LAYUP = 75
	DRIVING_FLOATING_BANK_JUMP_SHOT = 102
	DRIVING_FLOATING_JUMP_SHOT = 101
	DRIVING_LAYUP = 6
	DRIVING_REVERSE_LAYUP = 73
	DUNK = 7
	FADEAWAY_JUMPER = 63
	LAYUP = 5
	PULLUP_JUMP_SHOT = 79
	PUTBACK_LAYUP = 72
	RUNNING_FINGER_ROLL_LAYUP = 76
	RUNNING_JUMP_SHOT = 2
	STEP_BACK_JUMP_SHOT = 80
	TURNAROUND_BANK_SHOT = 85
	TURNAROUND_FADEAWAY_SHOT = 86
	TURNAROUND_JUMP_SHOT = 47


## Blocks

In [28]:
# Blocks are not included in the event feed but are a part of the EVENTMSGTYPE 2
import re
import operator

print('-----------------')

p = re.compile('(\s{2}|\' )([\w+ ]*)')

plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

description = ''
event_msg_action_types = {}

for play in plays:
  if play['EVENTMSGTYPE'] == 2:
    match = list()

    if play['HOMEDESCRIPTION'] is not None:
      match = p.findall(play['HOMEDESCRIPTION'])

      # looking for blocks
      if len(match) & (play['VISITORDESCRIPTION'] is not None):
        print(play['VISITORDESCRIPTION'])
    
    if not match and play['VISITORDESCRIPTION'] is not None:
      match = p.findall(play['VISITORDESCRIPTION'])

      # looking for blocks
      if len(match) & (play['HOMEDESCRIPTION'] is not None):
        print(play['HOMEDESCRIPTION'])
    
    if len(match):
      event_msg_action = re.sub(' ', '_', match[0][1]).upper()
      event_msg_action_types[event_msg_action] = play['EVENTMSGACTIONTYPE']

event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

print('-----------------')



-----------------
Turner BLOCK (1 BLK)
Turner BLOCK (2 BLK)
Gueye BLOCK (1 BLK)
Gueye BLOCK (2 BLK)
Griffin BLOCK (1 BLK)
Gueye BLOCK (3 BLK)
Turner BLOCK (3 BLK)
Turner BLOCK (4 BLK)
Haliburton BLOCK (1 BLK)
Mathews BLOCK (1 BLK)
Smith BLOCK (1 BLK)
-----------------
